# 1. 필요한 라이브러리

In [1]:
import os, sys
import requests

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [3]:
from common.config import get_config_info

# 2. 설정 파일 읽어오기

In [4]:
config = get_config_info('./config/settings2.json')

ENDPOINT = config['vr_endpoint']
MODEL = config['vr_api_model']
API_KEY = config['vr_api_key']
API_VERSION = config['vr_api_version']
INDEX_NAME = config['vr_index_name']
VIDEO_SAS_URL = config['vr_sas_url']
VIDEO_SAS_TOKEN = config['vr_sas_token']

# 3. 레퍼런스
[문서] https://learn.microsoft.com/ko-kr/azure/ai-services/computer-vision/how-to/video-retrieval

# 4. 인덱스 만들기

In [ ]:
def request_video_create_index():
    query_params = {
        'api-version': API_VERSION
    }
    
    endpoint = '{}{}{}'.format(ENDPOINT, MODEL, INDEX_NAME)

    # method
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY,
        'Content-Type': 'application/json'
    }

    body = {
        'metadataSchema': {
            'language': 'ko',
            'fields': [
                {
                'name': 'cameraId',
                'searchable': False,
                'filterable': True,
                'type': 'string'
                },
                {
                'name': 'timestamp',
                'searchable': False,
                'filterable': True,
                'type': 'datetime'
                }
            ]
        },
        'features': [
            {
                'name': 'vision',
                'domain': 'surveillance'
            },
            {
                'name': 'speech'
            }
        ]
    }
    
    response = requests.put(endpoint, params=query_params, headers=headers, json=body)

    if response.status_code == 201:
        print('success {}'.format(response.status_code))
    else:
        print('failure {}'.format(response.status_code))
    
    return response.json()

In [6]:
response1 = request_video_create_index()
response1

success 


{'name': '6b018-news3',
 'metadataSchema': {'language': 'ko',
  'fields': [{'name': 'cameraid',
    'searchable': False,
    'filterable': True,
    'type': 'string'},
   {'name': 'timestamp',
    'searchable': False,
    'filterable': True,
    'type': 'datetime'}]},
 'userData': {},
 'features': [{'name': 'vision',
   'modelVersion': '2023-05-31',
   'domain': 'surveillance'},
  {'name': 'speech', 'modelVersion': '2023-06-30', 'domain': 'generic'}],
 'eTag': '"a42d9770d14e434e9acbbe9e321541f3"',
 'createdDateTime': '2025-03-25T09:20:52.7651195Z',
 'lastModifiedDateTime': '2025-03-25T09:20:52.7651195Z'}

# 5. 비디오 업로드

In [7]:
def request_video_upload(ingestion_id):
    query_params = {
        'api-version': API_VERSION
    }
    
    endpoint = '{}{}{}/ingestions/{}'.format(ENDPOINT, MODEL, INDEX_NAME, ingestion_id)

    # method
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY,
        'Content-Type': 'application/json'
    }

    body = {
        'videos': [
            {
                'mode': 'add',
                'documentId': '02a504c9cd28296a8b74394ed7488045',
                'documentUrl': '{}?{}'.format(VIDEO_SAS_URL, VIDEO_SAS_TOKEN),
                'metadata': {
                    'cameraId': 'camera1',
                    'timestamp': '2025-03-25 17:20:33'
                }
            }
        ]
    }
    
    response = requests.put(endpoint, params=query_params, headers=headers, json=body)

    if response.status_code == 202:
        print('success {}'.format(response.status_code))
    else:
        print('failure {}'.format(response.status_code))
    
    return response.json()

In [8]:
response2 = request_video_upload(INDEX_NAME)
response2

success 202


{'name': '6b018-news3',
 'state': 'Running',
 'batchName': 'd9de43cb-ddad-4521-9ff7-fdab2b698f0b',
 'createdDateTime': '2025-03-25T09:21:51.4529927Z',
 'lastModifiedDateTime': '2025-03-25T09:21:52.0154954Z'}

# 6. 결과 상태 체크

In [9]:
def request_video_result():
    query_params = {
        'api-version': API_VERSION
    }
    
    endpoint = '{}{}{}/ingestions'.format(ENDPOINT, MODEL, INDEX_NAME)

    # method
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY,
        'Content-Type': 'application/json'
    }
    
    response = requests.get(endpoint, params=query_params, headers=headers)

    if response.status_code == 200:
        print('success {}'.format(response.status_code))
    else:
        print('failure {}'.format(response.status_code))
    
    return response.json()

In [10]:
response3 = request_video_result()
response3

success 200


{'value': [{'name': '6b018-news3',
   'state': 'Running',
   'batchName': 'd9de43cb-ddad-4521-9ff7-fdab2b698f0b',
   'createdDateTime': '2025-03-25T09:21:51.4529927Z',
   'lastModifiedDateTime': '2025-03-25T09:21:52.0154954Z'}]}

In [13]:
response3 = request_video_result()
response3

success 200


{'value': [{'name': '6b018-news3',
   'state': 'Completed',
   'batchName': 'd9de43cb-ddad-4521-9ff7-fdab2b698f0b',
   'createdDateTime': '2025-03-25T09:21:51.4529927Z',
   'lastModifiedDateTime': '2025-03-25T09:22:49.6877682Z'}]}

In [14]:
print(response3['value'][0]['name'], response3['value'][0]['state'])

6b018-news3 Completed


# 7. 조회

In [15]:
def request_video_search(query):
    query_params = {
        'api-version': API_VERSION
    }
    
    endpoint = '{}{}{}:queryByText'.format(ENDPOINT, MODEL, INDEX_NAME)

    # method
    headers = {
        'Ocp-Apim-Subscription-Key': API_KEY,
        'Content-Type': 'application/json'
    }
    
    body = {
        'queryText': query,
        'filters': {
            'stringFilters': [
                {
                    'fieldName': 'cameraId',
                    'values': [
                        'camera1'
                    ]
                }
            ],
            'featureFilters': ['vision'
            ]
        }
    }

    response = requests.post(endpoint, params=query_params, headers=headers, json=body)

    if response.status_code == 200:
        print('success {}'.format(response.status_code))
    else:
        print('failure {}'.format(response.status_code))
    
    return response.json()

In [16]:
response4 = request_video_search('sinkhole')
response4

success 200


{'value': [{'documentId': '02a504c9cd28296a8b74394ed7488045',
   'documentKind': 'VideoInterval',
   'start': '00:02:30',
   'end': '00:02:53',
   'best': '00:02:41',
   'relevance': 0.29472342133522034},
  {'documentId': '02a504c9cd28296a8b74394ed7488045',
   'documentKind': 'VideoInterval',
   'start': '00:02:53',
   'end': '00:03:16',
   'best': '00:03:02',
   'relevance': 0.27422139048576355},
  {'documentId': '02a504c9cd28296a8b74394ed7488045',
   'documentKind': 'VideoInterval',
   'start': '00:01:53',
   'end': '00:02:09',
   'best': '00:02:01',
   'relevance': 0.25027212500572205},
  {'documentId': '02a504c9cd28296a8b74394ed7488045',
   'documentKind': 'VideoInterval',
   'start': '00:06:06',
   'end': '00:06:15',
   'best': '00:06:06',
   'relevance': 0.2447357326745987},
  {'documentId': '02a504c9cd28296a8b74394ed7488045',
   'documentKind': 'VideoInterval',
   'start': '00:01:36',
   'end': '00:01:53',
   'best': '00:01:49',
   'relevance': 0.24421446025371552},
  {'document